### Generador de Noticias usando Trigramas a partir de noticias extraídas por web scraping del diario Gestión

### Instalación de Librerías

In [1]:
!pip install pandas nltk tqdm

DEPRECATION: Loading egg at c:\programdata\miniconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


### Implementación Trigramas

In [13]:
import json
import re
import random
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
from tqdm import tqdm

# --------------------------------------------------
# 0. Descargar recursos NLTK necesarios (solo la primera vez)
# --------------------------------------------------
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# --------------------------------------------------
# 1. Función para limpiar y tokenizar cada texto
# --------------------------------------------------
def limpiar_y_tokenizar(texto: str) -> list[str]:
    """
    1) Convierte a minúsculas.
    2) Elimina caracteres que no sean letras, dígitos o espacios.
    3) Tokeniza usando nltk.word_tokenize (divide en palabras y puntuación por separado).
    4) Filtra tokens que no contengan al menos una letra.
    Devuelve la lista de tokens “limpios”.
    """
    # 1) Convertir a minúsculas
    texto = texto.lower()
    # 2) Eliminar todo lo que no sea letra, número o espacio
    texto = re.sub(r"[^a-záéíóúñü0-9\s]", " ", texto)
    # 3) Tokenizar
    tokens = word_tokenize(texto, language="spanish")
    # 4) Filtrar tokens muy cortos o que no tengan ninguna letra
    tokens = [tok for tok in tokens if re.search(r"[a-záéíóúñü]", tok)]
    return tokens

# --------------------------------------------------
# 2. Leer el JSON y construir lista de documentos tokenizados
# --------------------------------------------------
def cargar_y_tokenizar_json(ruta_json: str) -> list[list[str]]:
    """
    Lee un archivo JSON con la siguiente estructura (lista de objetos):
      [
        {
          "title": "...",
          "category": "...",
          "summit": "...",
          "description": "...",
          "date": "...",
          "autor": "...",
          "tags": "['Seguro Social', 'Estados Unidos']",
          "url": "..."
        },
        { ... },
        ...
      ]
    Extrae el campo 'description' de cada objeto, lo limpia y tokeniza.
    Devuelve una lista de listas de tokens (cada lista es un documento tokenizado).
    """
    # Cargar el JSON completo. Asumimos que es un array de objetos.
    with open(ruta_json, 'r', encoding='utf-8') as f:
        data = json.load(f)

    documentos = []
    for item in tqdm(data, desc="Tokenizando descripciones"):
        # Extraer descripción; si no existe o está vacío, la saltamos
        desc = item.get("description", "")
        # Convertir a str por si viniera nulo
        desc = str(desc)
        # Limpiar y tokenizar
        tokens = limpiar_y_tokenizar(desc)
        # Solo añadimos si tenemos al menos 3 tokens (para formar trigramas)
        if len(tokens) >= 3:
            documentos.append(tokens)

    return documentos

# --------------------------------------------------
# 3. Construir el diccionario de trigramas y sus frecuencias
# --------------------------------------------------
def construir_trigramas(documentos: list[list[str]]) -> Counter:
    """
    Recorre cada documento (lista de tokens) y extrae todos los trigramas
    consecutivos (palabra_i, palabra_{i+1}, palabra_{i+2}).
    Devuelve un Counter que mapea cada trigram (tupla de 3 tokens) a su frecuencia total.
    """
    contador_trigramas = Counter()
    for doc in documentos:
        for i in range(len(doc) - 2):
            trigram = (doc[i], doc[i + 1], doc[i + 2])
            contador_trigramas[trigram] += 1
    return contador_trigramas

# --------------------------------------------------
# 4. Extraer los trigramas más frecuentes
# --------------------------------------------------
def obtener_trigramas_top(contador: Counter, top_n: int = 20) -> list[tuple[tuple[str, str, str], int]]:
    """
    Dado un Counter de trigramas, devuelve los `top_n` (trigrama, frecuencia)
    ordenados de mayor a menor frecuencia.
    """
    return contador.most_common(top_n)

# --------------------------------------------------
# 5. Construir estructuras auxiliares para generación de texto
# --------------------------------------------------
def construir_modelo_trigramas(contador: Counter) -> dict[tuple[str, str], dict[str, int]]:
    """
    A partir del Counter global de trigramas:
    Creamos un diccionario que para cada bigram (palabra_i, palabra_{i+1})
    tenga mapeado un diccionario con {palabra_{i+2}: frecuencia}.
    Estructura final: 
      modelo[(w1, w2)] = { w3a: freq1, w3b: freq2, … }
    """
    modelo = defaultdict(lambda: defaultdict(int))
    for (w1, w2, w3), freq in contador.items():
        modelo[(w1, w2)][w3] = freq
    return modelo

def muestrear_siguiente_palabra(probs: dict[str, int]) -> str:
    """
    Dado un diccionario de frecuencias {palabra: cuenta}, convertimos en lista de
    probabilidades y muestreamos una palabra aleatoria de acuerdo a su peso.
    """
    total = sum(probs.values())
    palabras = []
    pesos = []
    for palabra, cuenta in probs.items():
        palabras.append(palabra)
        pesos.append(cuenta / total)
    return random.choices(palabras, weights=pesos, k=1)[0]

# --------------------------------------------------
# 6. Generador de texto basado en trigramas
# --------------------------------------------------
def generar_texto_trigramas(modelo: dict[tuple[str, str], dict[str, int]],
                            inicio: tuple[str, str],
                            longitud: int = 50) -> str:
    """
    1) Partimos de un bigram (w1, w2) que debe existir en las claves del modelo.
    2) Vamos generando palabra a palabra: la siguiente w_{i+2} se muestrea con base en 
       modelo[(w1, w2)].
    3) La ventana de bigram “avanza” un paso: (w2, w_{i+2}), y seguimos así hasta llegar
       a `longitud` palabras generadas (excluyendo las dos iniciales).
    4) Devolvemos el texto generado concatenando con espacios.
    """
    w1, w2 = inicio
    if (w1, w2) not in modelo:
        raise ValueError(f"El par inicial {inicio} no está en el vocabulario de bigramas.")

    resultado = [w1, w2]
    for _ in range(longitud - 2):
        siguiente_dict = modelo.get((w1, w2), None)
        if not siguiente_dict:
            break  # Si no hay continuación, detenemos la generación
        w3 = muestrear_siguiente_palabra(siguiente_dict)
        resultado.append(w3)
        # Avanzamos la ventana
        w1, w2 = w2, w3

    return " ".join(resultado)

### Generarador de Noticias

In [14]:
# Cambiamos la ruta para apuntar a un JSON en lugar de CSV
RUTA_JSON = "gestionspider5.json"  # Ajusta este nombre a tu archivo real
TOP_N = 20          # Cantidad de trigramas más frecuentes a mostrar
LONGITUD_GEN = 50   # Número total de tokens que queremos generar (incluye dos palabras iniciales)

# 1) Leer y tokenizar desde JSON (campo "description")
documentos = cargar_y_tokenizar_json(RUTA_JSON)

# 2) Contar trigramas en todo el corpus
contador_tri = construir_trigramas(documentos)

# 3) Identeficar los TOP_N trigramas más comunes
top_trigramas = obtener_trigramas_top(contador_tri, TOP_N)

# 4) Construir el modelo auxiliar (bigram → {tercera palabra: frecuencia})
modelo_tri = construir_modelo_trigramas(contador_tri)

# 5) Elegir uno de los trigramas más frecuentes como “semilla” (bigram)
trigram_ejemplo, _ = top_trigramas[0]  # usamos el trigram más frecuente
bigram_inicial = (trigram_ejemplo[0], trigram_ejemplo[1])
print(f"\nUsando el bigram inicial: {bigram_inicial}\n")

# 6) Generar texto de ejemplo
texto_generado = generar_texto_trigramas(modelo_tri, bigram_inicial, longitud=LONGITUD_GEN)
print("Texto de ejemplo generado (primeras 50 palabras aprox.):\n")
print(texto_generado, "\n")

Tokenizando descripciones: 100%|██████████| 1308/1308 [00:01<00:00, 929.16it/s] 



Usando el bigram inicial: ('a', 'través')

Texto de ejemplo generado (primeras 50 palabras aprox.):

a través del abastecimiento en línea con la primera semana de cuatro nuevos proyectos para este jueves de abril la variación de las gratificaciones es inconstitucional pero usted sabe los ciudadanos cómo pueden actuar dentro de la región que a partir de las tres últimas plataformas del nuevo centro de 



### Combinaciones más frecuentes y su interpretación

In [15]:
print(f"\nLos {TOP_N} trigramas más frecuentes y sus frecuencias:\n")
for trigram, freq in top_trigramas:
    print(f"  {' '.join(trigram):<40}  →  {freq}")


Los 20 trigramas más frecuentes y sus frecuencias:

  a través de                               →  274
  uno de los                                →  215
  el ministerio de                          →  209
  de estados unidos                         →  204
  de la república                           →  192
  en el país                                →  192
  en el mercado                             →  164
  de us millones                            →  156
  en el perú                                →  155
  por lo que                                →  153
  de acuerdo con                            →  147
  la comisión de                            →  140
  de la empresa                             →  138
  por su parte                              →  135
  de millones de                            →  129
  en los últimos                            →  126
  una de las                                →  125
  en el caso                                →  124
  de este año                

### Interpretación

Top 1: frase “a través de” (274), frase muy común en redacción periodística para indicar el medio o canal por el cual algo ocurre, su alta frecuencia (274) sugiere que el corpus incluye numerosas notas en las que se cita qué “vehículo” o “medio” se utiliza para llevar a cabo alguna acción (por ejemplo, programas, acuerdos, pagos, transmisiones).

Top 2: frase “uno de los” (215), señala que, con frecuencia, las noticias sacan a relucir rankings, listas o comparaciones (“uno de los más relevantes”).

Top 3: frase “el ministerio de” (209), muy común cuando se menciona la actuación o comunicado de una cartera estatal, indica cobertura de políticas, normas o reportes oficiales.